In [1]:
import pandas as pd
import geopandas as gpd
import os, sys
import numpy as np
from matplotlib import pyplot as plt



In [ ]:
## look at hurrican wilma

In [7]:
# slosh = 'SLOSH_Innundation_Data/Inundation_polygons/Polygon_Cat1_outline.shp'
# gdf = gpd.read_file(slosh)
# gdf.plot()

In [8]:
# neighborhood csv... join regionID from neighborhood polygons to medianHousePricePerSqFt csv file
#neigh_shp = 'Neighborhoods/Neighborhood_polygons/PanhandleFl_Neigh_proj.shp'
zillow_csv = 'Zillow_Data/Zillow_Neighborhood_Data/Neighborhood_MedianValuePerSqft_AllHomes.csv'
neigh_shp = 'Neighborhoods/PanFl_Neigh_WithDistance.shp'

neigh_df = gpd.read_file(neigh_shp)
zillow_df = pd.read_csv(zillow_csv)

In [9]:
neigh_df.head()

,State,County,City,Name,RegionID,NEAR_FID,NEAR_DIST,geometry
0,FL,Orange,Orlando,Alafaya,15046,33846,20353.791610,"POLYGON ((1455291.559658508 714284.254792735, ..."
1,FL,Broward,Hollywood,Hillcrest,25125,34917,4363.193135,"POLYGON ((1589487.493421007 459691.2616779506,..."
2,FL,Miami-Dade,Miami,Brickell,55477,34997,0.000000,"POLYGON ((1593401.634173049 435138.7320546452,..."
3,FL,Pasco,New Port Richey,North Park,116942,5201,488.849583,"POLYGON ((1303881.935526199 667939.7120311093,..."
4,FL,Volusia,New Smyrna Beach,South West,121790,33325,286.567972,"POLYGON ((1463240.95545853 776744.8219052199, ..."


In [10]:
zillow_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09
0,274772,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas,1,70.0,71.0,71.0,...,161,163,166,169,170,172,173,173,174,175
1,192689,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark,2,74.0,74.0,74.0,...,139,141,144,145,147,149,151,153,155,157
2,270958,Upper West Side,New York,NY,New York-Newark-Jersey City,New York,3,285.0,287.0,289.0,...,1508,1503,1505,1504,1507,1514,1511,1496,1487,1485
3,118208,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,4,96.0,96.0,97.0,...,348,353,358,362,363,365,367,369,371,374
4,270957,Upper East Side,New York,NY,New York-Newark-Jersey City,New York,5,326.0,326.0,327.0,...,1401,1398,1402,1405,1412,1422,1421,1407,1396,1390


In [12]:
## see how many regionIDs are in neigh_df
neigh_df['RegionID'].unique().shape

(1050,)

In [24]:
# join zillow_df to neigh_df by RegionID

# help(neigh_df.join)
join_df = neigh_df.join(zillow_df, on='RegionID', how='left', lsuffix='_nbrhd')

In [23]:
join_df.shape, neigh_df.shape, zillow_df.shape

((0, 285), (1050, 8), (7538, 277))

In [32]:
neigh_df['RegionID'].dtype, zillow_df['RegionID'].dtype

(dtype('O'), dtype('int64'))

In [33]:
## the data type of RegionID is Object for neighborhood, and Int64. need to change data type of RegionID column in neigh
neigh_df[['RegionID']] = neigh_df[['RegionID']].apply(pd.to_numeric)
# neigh_df[['RegionID']] = neigh_df[['RegionID']].astype('int64') # this would explicitly change it to int64
neigh_df['RegionID'].dtype

dtype('int64')

In [40]:
## now we can join the dataframes by RegionID
join_df = neigh_df.join(zillow_df, on='RegionID', how='outer', lsuffix='_nbrhd')
join_df.shape, neigh_df.shape, zillow_df.shape

((8588, 285), (1050, 8), (7538, 277))

In [39]:
neigh_df['RegionID'].dtype, zillow_df['RegionID'].dtype

(dtype('int64'), dtype('int64'))

In [42]:
join_df.tail()

,State_nbrhd,County,City_nbrhd,Name,RegionID_nbrhd,NEAR_FID,NEAR_DIST,geometry,RegionID,RegionName,...,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09
1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7533,Sunridge Park,...,205.0,207.0,208.0,209.0,209.0,209.0,209.0,208.0,208.0,208.0
1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7534,Andreas Hills,...,348.0,348.0,344.0,338.0,336.0,336.0,336.0,336.0,337.0,338.0
1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7535,Little Tuscany,...,328.0,332.0,336.0,340.0,343.0,346.0,348.0,351.0,354.0,357.0
1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7536,North Village,...,138.0,139.0,139.0,139.0,140.0,141.0,141.0,142.0,142.0,142.0
1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7537,Woodhaven,...,209.0,209.0,209.0,210.0,210.0,211.0,210.0,209.0,209.0,211.0
